# Zad 6

Utworzyć tabelę OrdersToProcess(SalesOrderID INT, Delayed BIT), w której będą przechowywane zamówienia jeszcze nie dostarczone, a flaga Delayed określa czy DueDate nie został przekroczony. Przygotować zapytanie, które zaktualizuje tę tabelę w oparciu o tabelę SalesOrderHeader i należy skorzystać z konstrukcji MERGE. Aby przetestować działanie tej metody, warto dogenerować trochę danych (w bazie testowej jest ich niewiele).

Check if db exists -\> create if not

In [1]:
IF NOT EXISTS (SELECT * from sysobjects where name='SalesLT.OrdersToProcess' and xtype='U')
    CREATE TABLE SalesLT.OrdersToProcess (
        SalesOrderID INT, 
        Delayed BIT
    );
go

: Msg 2714, Level 16, State 6, Line 2
There is already an object named 'OrdersToProcess' in the database.

Total execution time: 00:00:00.110

## Exploration

In [8]:
SELECT TOP 5 *
FROM SalesLT.SalesOrderHeader

(5 rows affected)

Total execution time: 00:00:00.056

SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,ShipToAddressID,BillToAddressID,ShipMethod,CreditCardApprovalCode,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
71774,2,2008-06-01 00:00:00.000,2008-06-13 00:00:00.000,2008-06-08 00:00:00.000,5,0,SO71774,PO348186287,10-4020-000609,29847,1092,1092,CARGO TRANSPORT 5,NULL,"880,3484","70,4279","22,0087","972,785",NULL,89e42cdc-8506-48a2-b89b-eb3e64e3554e,2008-06-08 00:00:00.000
71776,2,2008-06-01 00:00:00.000,2008-06-13 00:00:00.000,2008-06-08 00:00:00.000,5,0,SO71776,PO19952192051,10-4020-000106,30072,640,640,CARGO TRANSPORT 5,NULL,"78,81","6,3048","1,9703","87,0851",NULL,8a3448c5-e677-4158-a29b-dd33069be0b0,2008-06-08 00:00:00.000
71780,2,2008-06-01 00:00:00.000,2008-06-13 00:00:00.000,2008-06-08 00:00:00.000,5,0,SO71780,PO19604173239,10-4020-000340,30113,653,653,CARGO TRANSPORT 5,NULL,"38418,6895","3073,4952","960,4672","42452,6519",NULL,a47665d2-7ac9-4cf3-8a8b-2a3883554284,2008-06-08 00:00:00.000
71782,2,2008-06-01 00:00:00.000,2008-06-13 00:00:00.000,2008-06-08 00:00:00.000,5,0,SO71782,PO19372114749,10-4020-000582,29485,1086,1086,CARGO TRANSPORT 5,NULL,"39785,3304","3182,8264","994,6333","43962,7901",NULL,f1be45a5-5c57-4a50-93c6-5f8be44cb7cb,2008-06-08 00:00:00.000
71783,2,2008-06-01 00:00:00.000,2008-06-13 00:00:00.000,2008-06-08 00:00:00.000,5,0,SO71783,PO19343113609,10-4020-000024,29957,992,992,CARGO TRANSPORT 5,NULL,"83858,4261","6708,6741","2096,4607","92663,5609",NULL,7db2329e-6446-42a8-8915-9c8370b68ed8,2008-06-08 00:00:00.000


In [12]:
exec sp_help "SalesLT.SalesOrderHeader"

No views with schema binding reference table 'SalesLT.SalesOrderHeader'.

Total execution time: 00:00:03.023

Name,Owner,Type,Created_datetime
SalesOrderHeader,dbo,user table,2022-03-07 12:23:11.100


Column_name,Type,Computed,Length,Prec,Scale,Nullable,TrimTrailingBlanks,FixedLenNullInSource,Collation
SalesOrderID,int,no,4,10,0,no,(n/a),(n/a),NULL
RevisionNumber,tinyint,no,1,3,0,no,(n/a),(n/a),NULL
OrderDate,datetime,no,8,,,no,(n/a),(n/a),NULL
DueDate,datetime,no,8,,,no,(n/a),(n/a),NULL
ShipDate,datetime,no,8,,,yes,(n/a),(n/a),NULL
Status,tinyint,no,1,3,0,no,(n/a),(n/a),NULL
OnlineOrderFlag,Flag,no,1,,,no,(n/a),(n/a),NULL
SalesOrderNumber,nvarchar,yes,50,,,no,(n/a),(n/a),SQL_Latin1_General_CP1_CI_AS
PurchaseOrderNumber,OrderNumber,no,50,,,yes,(n/a),(n/a),SQL_Latin1_General_CP1_CI_AS
AccountNumber,AccountNumber,no,30,,,yes,(n/a),(n/a),SQL_Latin1_General_CP1_CI_AS


Identity,Seed,Increment,Not For Replication
No identity column defined.,NULL,NULL,NULL


RowGuidCol
No rowguidcol column defined.


Data_located_on_filegroup
PRIMARY


index_name,index_description,index_keys
AK_SalesOrderHeader_rowguid,"nonclustered, unique, unique key",rowguid
AK_SalesOrderHeader_SalesOrderNumber,"nonclustered, unique, unique key",SalesOrderNumber
IX_SalesOrderHeader_CustomerID,nonclustered,CustomerID
PK_SalesOrderHeader_SalesOrderID,"clustered, unique, primary key",SalesOrderID


constraint_type,constraint_name,delete_action,update_action,status_enabled,status_for_replication,constraint_keys
UNIQUE (non-clustered),AK_SalesOrderHeader_rowguid,(n/a),(n/a),(n/a),(n/a),rowguid
UNIQUE (non-clustered),AK_SalesOrderHeader_SalesOrderNumber,(n/a),(n/a),(n/a),(n/a),SalesOrderNumber
CHECK Table Level,CK_SalesOrderHeader_DueDate,(n/a),(n/a),Enabled,Is_For_Replication,([DueDate]>=[OrderDate])
CHECK on column Freight,CK_SalesOrderHeader_Freight,(n/a),(n/a),Enabled,Is_For_Replication,([Freight]>=(0.00))
CHECK Table Level,CK_SalesOrderHeader_ShipDate,(n/a),(n/a),Enabled,Is_For_Replication,([ShipDate]>=[OrderDate] OR [ShipDate] IS NULL)
CHECK on column Status,CK_SalesOrderHeader_Status,(n/a),(n/a),Enabled,Is_For_Replication,([Status]>=(0) AND [Status]<=(8))
CHECK on column SubTotal,CK_SalesOrderHeader_SubTotal,(n/a),(n/a),Enabled,Is_For_Replication,([SubTotal]>=(0.00))
CHECK on column TaxAmt,CK_SalesOrderHeader_TaxAmt,(n/a),(n/a),Enabled,Is_For_Replication,([TaxAmt]>=(0.00))
DEFAULT on column Freight,DF_SalesOrderHeader_Freight,(n/a),(n/a),(n/a),(n/a),((0.00))
DEFAULT on column ModifiedDate,DF_SalesOrderHeader_ModifiedDate,(n/a),(n/a),(n/a),(n/a),(getdate())


Table is referenced by foreign key
kpabd-test.SalesLT.SalesOrderDetail: FK_SalesOrderDetail_SalesOrderHeader_SalesOrderID


## Merge update

In [21]:
ALTER TABLE SalesLT.SalesOrderHeader
NOCHECK CONSTRAINT CK_SalesOrderHeader_ShipDate;  
GO  

Commands completed successfully.

Total execution time: 00:00:00.076

In [23]:
ALTER TABLE SalesLT.SalesOrderHeader
NOCHECK CONSTRAINT ALL
GO  

Commands completed successfully.

Total execution time: 00:00:00.109

In [140]:
INSERT INTO SalesLT.SalesOrderHeader(DueDate, ShipDate, Comment, CustomerID, ShipMethod)
VALUES
    ('2008-06-13 00:00:00.000', NULL, 'testingval', 29847, 'CARGO METHOD 5'),
    ('2008-06-11 00:00:00.000', NULL, 'testingval', 29847, 'CARGO METHOD 5'),
    ('2014-06-12 00:00:00.000', NULL, 'testingval', 29847, 'CARGO METHOD 5'),
    ('2022-06-12 00:00:00.000', NULL, 'testingval', 29847, 'CARGO METHOD 5'),
    ('2023-06-12 00:00:00.000', NULL, 'testingval', 29847, 'CARGO METHOD 5'),
    ('2022-03-13 00:00:00.000', NULL, 'testingval', 29847, 'CARGO METHOD 5')


(6 rows affected)

Total execution time: 00:00:00.058

In [141]:
SELECT TOP 10 *
FROM SalesLT.SalesOrderHeader
WHERE DATEDIFF(hour, DueDate, GETDATE()) > 0 AND SalesLT.SalesOrderHeader.Status BETWEEN 1 AND 4

(10 rows affected)

Total execution time: 00:00:00.063

SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,ShipToAddressID,BillToAddressID,ShipMethod,CreditCardApprovalCode,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
12,0,2022-03-13 17:03:34.593,2008-06-13 00:00:00.000,NULL,1,1,SO12,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,975f5498-cf27-49a5-bf20-c6b29e7399b0,2022-03-13 17:03:34.593
13,0,2022-03-13 17:03:34.593,2008-06-11 00:00:00.000,NULL,1,1,SO13,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,a4b8356c-c97b-4a48-83d8-8bd2b597daab,2022-03-13 17:03:34.593
14,0,2022-03-13 17:03:34.593,2008-06-12 00:00:00.000,NULL,1,1,SO14,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,6cb40b3d-237d-4ca7-a817-2ce77e90135d,2022-03-13 17:03:34.593
15,0,2022-03-13 17:23:21.790,2008-06-13 00:00:00.000,NULL,1,1,SO15,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,e2c8bc1b-9443-45aa-b144-cfd51a9b71d7,2022-03-13 17:23:21.790
16,0,2022-03-13 17:23:21.790,2008-06-11 00:00:00.000,NULL,1,1,SO16,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,e899aaad-63ee-49fd-96d7-7ff36e585ca3,2022-03-13 17:23:21.790
17,0,2022-03-13 17:23:21.790,2014-06-12 00:00:00.000,NULL,1,1,SO17,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,894e41e7-4999-4236-a49e-a0985772b573,2022-03-13 17:23:21.790
20,0,2022-03-13 18:29:46.703,2008-06-13 00:00:00.000,NULL,1,1,SO20,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,5b8f9641-5c31-4a78-9d63-c6e118f57f2f,2022-03-13 18:29:46.703
21,0,2022-03-13 18:29:46.703,2008-06-11 00:00:00.000,NULL,1,1,SO21,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,33ea5b8b-8e09-454e-9568-bd83096f0e5d,2022-03-13 18:29:46.703
22,0,2022-03-13 18:29:46.703,2014-06-12 00:00:00.000,NULL,1,1,SO22,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,337aa653-aa9c-481f-af23-85fc66a229d8,2022-03-13 18:29:46.703
25,0,2022-03-13 18:31:45.017,2008-06-13 00:00:00.000,NULL,1,1,SO25,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,86001f11-fda9-4eb2-923c-2b7cd6b310ff,2022-03-13 18:31:45.017


MERGE -\> INSERT, UPDATE, DELETE in one

https://www.sqlservertutorial.net/sql-server-basics/sql-server-merge/

In [142]:
MERGE SalesLT.OrdersToProcess USING SalesLT.SalesOrderHeader
ON (SalesLT.OrdersToProcess.SalesOrderID = SalesLT.SalesOrderHeader.SalesOrderID)
WHEN MATCHED AND (SalesLT.SalesOrderHeader.Status BETWEEN 1 AND 4)
    THEN UPDATE SET
        SalesLT.OrdersToProcess.Delayed = IIF (DATEDIFF(hour, DueDate, GETDATE()) > 0, 1, 0)
WHEN NOT MATCHED BY TARGET AND (SalesLT.SalesOrderHeader.Status BETWEEN 1 AND 4)
    THEN INSERT (SalesOrderID, Delayed)
        VALUES (SalesLT.SalesOrderHeader.SalesOrderID, IIF (DATEDIFF(hour, DueDate, GETDATE()) > 0, 1, 0))
WHEN NOT MATCHED BY SOURCE
    THEN DELETE;

(30 rows affected)

Total execution time: 00:00:00.065

In [137]:
SELECT * 
FROM SalesLT.OrdersToProcess
JOIN SalesLT.SalesOrderHeader ON SalesLt.SalesOrderHeader.SalesOrderID = SalesLT.OrdersToProcess.SalesOrderID

(24 rows affected)

Total execution time: 00:00:00.126

SalesOrderID,Delayed,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,ShipToAddressID,BillToAddressID,ShipMethod,CreditCardApprovalCode,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
12,1,12,0,2022-03-13 17:03:34.593,2008-06-13 00:00:00.000,NULL,1,1,SO12,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,975f5498-cf27-49a5-bf20-c6b29e7399b0,2022-03-13 17:03:34.593
13,1,13,0,2022-03-13 17:03:34.593,2008-06-11 00:00:00.000,NULL,1,1,SO13,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,a4b8356c-c97b-4a48-83d8-8bd2b597daab,2022-03-13 17:03:34.593
14,1,14,0,2022-03-13 17:03:34.593,2008-06-12 00:00:00.000,NULL,1,1,SO14,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,6cb40b3d-237d-4ca7-a817-2ce77e90135d,2022-03-13 17:03:34.593
15,1,15,0,2022-03-13 17:23:21.790,2008-06-13 00:00:00.000,NULL,1,1,SO15,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,e2c8bc1b-9443-45aa-b144-cfd51a9b71d7,2022-03-13 17:23:21.790
16,1,16,0,2022-03-13 17:23:21.790,2008-06-11 00:00:00.000,NULL,1,1,SO16,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,e899aaad-63ee-49fd-96d7-7ff36e585ca3,2022-03-13 17:23:21.790
17,1,17,0,2022-03-13 17:23:21.790,2014-06-12 00:00:00.000,NULL,1,1,SO17,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,894e41e7-4999-4236-a49e-a0985772b573,2022-03-13 17:23:21.790
18,0,18,0,2022-03-13 17:23:21.790,2022-06-12 00:00:00.000,NULL,1,1,SO18,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,a4fda8f0-8f62-415e-93e0-f088b286c3bc,2022-03-13 17:23:21.790
19,0,19,0,2022-03-13 17:23:21.790,2023-06-12 00:00:00.000,NULL,1,1,SO19,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,97271a59-5845-43d9-9eb7-4e3b9314e712,2022-03-13 17:23:21.790
20,1,20,0,2022-03-13 18:29:46.703,2008-06-13 00:00:00.000,NULL,1,1,SO20,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,5b8f9641-5c31-4a78-9d63-c6e118f57f2f,2022-03-13 18:29:46.703
21,1,21,0,2022-03-13 18:29:46.703,2008-06-11 00:00:00.000,NULL,1,1,SO21,NULL,NULL,29847,NULL,NULL,CARGO METHOD 5,NULL,"0,00","0,00","0,00","0,00",testingval,33ea5b8b-8e09-454e-9568-bd83096f0e5d,2022-03-13 18:29:46.703


## Cleanup

In [2]:
DELETE SalesLT.SalesOrderHeader
WHERE Comment = 'testingval'

(30 rows affected)

Total execution time: 00:00:00.060

In [3]:
ALTER TABLE SalesLT.SalesOrderHeader
WITH CHECK CHECK CONSTRAINT ALL;
GO  

Commands completed successfully.

Total execution time: 00:00:00.072

In [138]:
DROP TABLE SalesLT.OrdersToProcess

Commands completed successfully.

Total execution time: 00:00:00.071